In [36]:
from dolfinx import fem, mesh, io
from ufl import TrialFunction, TestFunction, inner, dot, grad, div, dx, ds, sym, Identity
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np

# Print version
import dolfinx
print(f"DOLFINx version: {dolfinx.__version__}")

# Parameters
L = 1.0  # Domain size (square [0,1] x [0,1])
E = 10e6  # Young's modulus (Pa)
nu = 0.2  # Poisson's ratio
B = 0.8  # Biot coefficient
M = 1e9  # Biot modulus (Pa)
k = 1e-12  # Permeability (m^2)
mu = 1e-3  # Fluid viscosity (Pa·s)
F = 1e6  # Compressive traction (Pa)
T = 1000.0  # Final time (s)
num_steps = 50  # Number of time steps
dt = T / num_steps  # Time step size
n = 32  # Mesh resolution (nxn)

# Material properties (isotropic elasticity)
lambda_ = E * nu / ((1 + nu) * (1 - 2 * nu))
mu_ = E / (2 * (1 + nu))  # Shear modulus
k_mu = k / mu  # Permeability/viscosity ratio

# Create mesh
domain = mesh.create_rectangle(
    MPI.COMM_WORLD, [[0.0, 0.0], [L, L]], [n, n], mesh.CellType.quadrilateral
)

# Define function spaces
V = fem.functionspace(domain, ("P", 2, (2,)))  # P2 for displacement
Q = fem.functionspace(domain, ("P", 1))        # P1 for pressure
print("P2 and P1 FunctionSpaces created successfully")

# Define functions
u = TrialFunction(V)
p = TrialFunction(Q)
v = TestFunction(V)
q = TestFunction(Q)
u_s = fem.Function(V)  # Current displacement
p_s = fem.Function(Q)  # Current pressure
u_n = fem.Function(V)  # Previous displacement
p_n = fem.Function(Q)  # Previous pressure

# Weak form
def sigma(u):
    epsilon = sym(grad(u))
    return lambda_ * div(u) * Identity(2) + 2 * mu_ * epsilon

# Mass balance form: a_pq (p, q) + a_uq (u, q) = L_q
a_pq = inner(q, (1 / M) * p / dt) * dx + inner(grad(q), k_mu * grad(p)) * dx
a_uq = inner(q, B * div(u / dt)) * dx
L_q = inner(q, (1 / M) * p_n / dt) * dx + inner(q, B * div(u_n / dt)) * dx

# Equilibrium form: a_uv (u, v) + a_pv (p, v) = L_v
a_uv = inner(sym(grad(v)), sigma(u)) * dx
a_pv = -inner(div(v), B * p) * dx
L_v = inner(v, fem.Constant(domain, (0.0, -F))) * ds

# Compile forms
a_pq = fem.form(a_pq)
a_uq = fem.form(a_uq)
a_uv = fem.form(a_uv)
a_pv = fem.form(a_pv)
L_q = fem.form(L_q)
L_v = fem.form(L_v)

# Block forms
a = [[a_uv, a_pv], [a_uq, a_pq]]
L = [L_v, L_q]

# Boundary conditions
tdim = domain.topology.dim
fdim = tdim - 1

def top_boundary(x):
    return np.abs(x[1] - 1.0) < 1e-10
facets_top = mesh.locate_entities_boundary(domain, fdim, top_boundary)

def bottom_boundary(x):
    return np.abs(x[1] - 0.0) < 1e-10
facets_bottom = mesh.locate_entities_boundary(domain, fdim, bottom_boundary)
dofs_bottom_uy = fem.locate_dofs_topological(V.sub(1), fdim, facets_bottom)
bc_bottom_uy = fem.dirichletbc(0.0, dofs_bottom_uy, V.sub(1))

def side_boundaries(x):
    return np.logical_or(np.abs(x[0] - 0.0) < 1e-10, np.abs(x[0] - 1.0) < 1e-10)
facets_sides = mesh.locate_entities_boundary(domain, fdim, side_boundaries)
dofs_sides_p = fem.locate_dofs_topological(Q, fdim, facets_sides)
bc_sides_p = fem.dirichletbc(0.0, dofs_sides_p, Q)

bcs = [bc_bottom_uy, bc_sides_p]

# Output setup
vtkfile_u = io.VTXWriter(domain.comm, "displacement.bp", [u_s], engine="BP5")
vtkfile_p = io.VTXWriter(domain.comm, "pressure.bp", [p_s], engine="BP5")

# Block solver setup
# Create block matrix and vector
A = fem.petsc.assemble_matrix_block(a, bcs=bcs)
A.assemble()
b = fem.petsc.assemble_vector_block(L, a, bcs=bcs)

# Create PETSc solver
ksp = PETSc.KSP().create(MPI.COMM_WORLD)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.setFromOptions()

# Create block solution vector
x = A.createVecLeft()

t = 0.0
for n in range(num_steps):
    t += dt
    print(f"Time step {n+1}/{num_steps}, t = {t:.3f}")

    # Assemble RHS
    b.zeroEntries()
    fem.petsc.assemble_vector_block(b, L, a, bcs=bcs)

    # Solve
    ksp.solve(b, x)

    # Distribute solution
    offset = V.dofmap.index_map.size_local * V.dofmap.index_map_bs
    u_s.x.array[:offset] = x.array_r[:offset]
    p_s.x.array[:] = x.array_r[offset:]

    # Save solutions
    vtkfile_u.write(t)
    vtkfile_p.write(t)

    # Update previous solution
    u_n.x.array[:] = u_s.x.array
    p_n.x.array[:] = p_s.x.array

# Clean up
vtkfile_u.close()
vtkfile_p.close()
ksp.destroy()
A.destroy()
b.destroy()
x.destroy()

print("Simulation completed!")

DOLFINx version: 0.9.0
P2 and P1 FunctionSpaces created successfully
Time step 1/50, t = 20.000
Time step 2/50, t = 40.000
Time step 3/50, t = 60.000
Time step 4/50, t = 80.000
Time step 5/50, t = 100.000
Time step 6/50, t = 120.000
Time step 7/50, t = 140.000
Time step 8/50, t = 160.000
Time step 9/50, t = 180.000
Time step 10/50, t = 200.000
Time step 11/50, t = 220.000
Time step 12/50, t = 240.000
Time step 13/50, t = 260.000
Time step 14/50, t = 280.000
Time step 15/50, t = 300.000
Time step 16/50, t = 320.000
Time step 17/50, t = 340.000
Time step 18/50, t = 360.000
Time step 19/50, t = 380.000
Time step 20/50, t = 400.000
Time step 21/50, t = 420.000
Time step 22/50, t = 440.000
Time step 23/50, t = 460.000
Time step 24/50, t = 480.000
Time step 25/50, t = 500.000
Time step 26/50, t = 520.000
Time step 27/50, t = 540.000
Time step 28/50, t = 560.000
Time step 29/50, t = 580.000
Time step 30/50, t = 600.000
Time step 31/50, t = 620.000
Time step 32/50, t = 640.000
Time step 33/50,